In this document, we will collect the data processing functions. The functions themselves will be stored within ../proj_mod/data_processing.py where proj_mod can be imported as a python module when needed. Following two python block surves as an example of importing proj_mod. 

In [1]:
import sys
sys.path.append("../")

import pandas as pd
import numpy as np

import time

import glob

# This forces Jupyter Notebook to reload the module instead of using the cached import
import importlib

from proj_mod import data_processing
importlib.reload(data_processing); #Adding ";" to suppress output.

## Book data harvesting function by stock and time id

We will first demonstrate the function book_for_stock(). Following function book_for_stock is inefficient, one should load the data by stock_id and "ignore" time_id. 

In [2]:
df_book_0_5=data_processing.book_for_stock(str_file_path="../raw_data/kaggle_ORVP/book_train.parquet",stock_id=0,time_id=5, create_para=True)

In [3]:
df_book_0_5

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap,log_return
0,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,1.001448,0.000014
1,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000000
2,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,-0.000005
3,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,0.000000
4,5,11,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,5,585,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0,1.003731,0.000245
297,5,586,1.003129,1.003749,1.002612,1.003801,100,3,2,3,0,1.003731,0.000000
298,5,587,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0,1.003731,0.000000
299,5,588,1.003129,1.003749,1.002612,1.003801,100,3,2,3,0,1.003731,0.000000


## Calculating RV by stock and time id. (This appears not very practical due to speed). 

In the following, we show an example of realized_vol function. Following function is inefficient, use create_df_RV_by_row_id_stock instead. 

In [4]:
rv, row= data_processing.realized_vol(df_book_0_5)

## Saving the RV values of current 10s to safe calculation time for future use. 

I created an alternative function to calculate values enmass, going through each df separately is extremely time-consuming. 

In [2]:
path_book="../raw_data/kaggle_ORVP/book_train.parquet"

In [3]:
start_time = time.time()
df_rv=data_processing.create_df_RV_by_row_id(path_book)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 62.43 seconds


In [4]:
df_rv

,time_id,RV,row_id,stock_id
0,5,0.002185,93-5,93
1,11,0.001205,93-11,93
2,16,0.001461,93-16,93
3,31,0.001693,93-31,93
4,62,0.001296,93-62,93
...,...,...,...,...
428927,32751,0.002337,104-32751,104
428928,32753,0.001500,104-32753,104
428929,32758,0.002272,104-32758,104
428930,32763,0.001949,104-32763,104


In [9]:
df_rv.to_csv("../processed_data/RV_by_row_id.csv",index=False)

## The calculation of RV values can be done in parallel now (this function also uses less memory after the initial spike):

In [5]:
start_time = time.time()
path_book="../raw_data/kaggle_ORVP/book_train.parquet"
df_rv_parallel=data_processing.create_df_RV_by_row_id_parallel(path_book)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 17.24 seconds


In [6]:
df_rv_parallel

,time_id,RV,row_id,stock_id
0,5,0.002185,93-5,93
1,11,0.001205,93-11,93
2,16,0.001461,93-16,93
3,31,0.001693,93-31,93
4,62,0.001296,93-62,93
...,...,...,...,...
428927,32751,0.002337,104-32751,104
428928,32753,0.001500,104-32753,104
428929,32758,0.002272,104-32758,104
428930,32763,0.001949,104-32763,104


## Trade data harvesting function by stock and time id. 

Function trade_for_stock() is similar. Following function book_for_stock is inefficient, one should load the data by stock_id and "ignore" time_id. 

In [24]:
df_trade_0_5=data_processing.trade_for_stock(str_file_path="../raw_data/kaggle_ORVP/trade_train.parquet",stock_id=0,time_id=5)

In [25]:
df_trade_0_5.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0


## Precalculate the trade data on avg and std of of price, size, order, and sum of size, and order. 

In [8]:
df_trade_vals=data_processing.create_df_trade_vals_by_row_id(str_path="../raw_data/kaggle_ORVP/trade_train.parquet")

In [9]:
df_trade_vals

,time_id,price_mean,price_std,size_sum,size_mean,size_std,order_count_sum,order_count_mean,order_count_std,row_id,stock_id
0,5,1.002227,0.001003,35728,140.109804,165.359986,815,3.196078,2.865898,93-5,93
1,11,1.000889,0.000439,23796,226.628571,242.518804,402,3.828571,3.861688,93-11,93
2,16,0.999648,0.000335,20642,231.932584,223.418268,330,3.707865,3.348006,93-16,93
3,31,0.999804,0.000393,12960,196.363636,193.771685,256,3.878788,2.836656,93-31,93
4,62,0.999488,0.000338,5547,84.045455,118.524705,178,2.696970,2.811896,93-62,93
...,...,...,...,...,...,...,...,...,...,...,...
428908,32751,0.999914,0.000487,8089,120.731343,150.944961,216,3.223881,3.024250,104-32751,104
428909,32753,0.999238,0.000271,7782,131.898305,164.677500,221,3.745763,3.826375,104-32753,104
428910,32758,0.999601,0.000453,2804,100.142857,114.605835,74,2.642857,3.188106,104-32758,104
428911,32763,0.999555,0.000570,24618,276.606742,280.412546,429,4.820225,4.187540,104-32763,104


In [57]:
df_trade_vals.to_csv("../processed_data/trade_vals_by_row_id.csv",index=False)

## Time series creation function

For each row id "a-b" (stock id a and time id b), we have the trade data. 
We create the RV of sub-intervals (e.g. seconds_in_bracket in interval [0,10]) for all disjoint sub-intervals within [0,600] (e.g. [0,10], [11, 21], ...). 
This will help us to bypass the fact that there are different total number of seconds_in_bracket in each row_id. 
This sequence of RV can serve as a time series data. 

In [2]:
book_0=pd.read_parquet("../raw_data/kaggle_ORVP/book_train.parquet/stock_id=0")

In [3]:
book_0["stock_id"]=0

In [4]:
book_0=data_processing.create_df_wap_logreturn(book_0)

In [5]:
book_0

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap,log_return,wap_mid,log_return_mid
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,1.001448,0.000014,1.002305,1.249354e-06
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000000,1.002330,2.517913e-05
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,-0.000005,1.002327,-3.203381e-06
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,0.000000,1.002327,0.000000e+00
5,5,11,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000005,1.002330,3.203381e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28,0,0.998515,0.000434,0.998427,1.618314e-04
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28,0,0.998516,0.000001,0.998239,-1.883461e-04
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28,0,0.998516,0.000000,0.998352,1.125510e-04
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28,0,0.998517,0.000001,0.998351,-2.465059e-07


Following function created one time_series feature for all time_id (3820 of them, each with 60 created subintervals) for stock_id==0 in 20s on my machine. Be advised about time when using. 

The construction of 'book_0' is now included in the function 'create_timeseries_stock' itself, as it function needs to be self-contained to be parallelizable with joblib

In [6]:
start_time = time.time()
path = "../raw_data/kaggle_ORVP/book_train.parquet/stock_id=0"
book_0_time=data_processing.create_timeseries_stock(path=path,
                                                    dict_agg={"log_return":data_processing.rv},
                                                    dict_rename={"log_return":"sub_int_RV"})
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 9.12 seconds


In [7]:
book_0_time

,time_id,sub_int_RV,sub_int_num,stock_id,row_id
0,5,1.498329e-05,1,0,0_5
1,11,3.839994e-06,1,0,0_11
2,16,4.321017e-04,1,0,0_16
3,31,0.000000e+00,1,0,0_31
4,62,2.345477e-04,1,0,0_62
...,...,...,...,...,...
213035,32736,6.959812e-04,60,0,0_32736
213036,32739,5.730293e-04,60,0,0_32739
213037,32748,7.492413e-05,60,0,0_32748
213038,32753,2.130664e-07,60,0,0_32753


In [8]:
book_0_time[book_0_time["time_id"]==5].head()

,time_id,sub_int_RV,sub_int_num,stock_id,row_id
0,5,0.000015,1,0,0_5
3648,5,0.000010,2,0,0_5
7230,5,0.001057,3,0,0_5
14416,5,0.000898,5,0,0_5
17996,5,0.000892,6,0,0_5


Now we can calculate the time series for the whole book. We have 112 stock files so it needs to be done in parallel.

In [9]:
start_time = time.time()
path_book="../raw_data/kaggle_ORVP/book_train.parquet"
book_time=data_processing.create_timeseries(path_book,
                                            dict_agg={"log_return":data_processing.rv},
                                            dict_rename={"log_return":"sub_int_RV"})
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 85.69 seconds


In [10]:
book_time[book_time['stock_id']==0]

,time_id,sub_int_RV,sub_int_num,stock_id,row_id
20935225,5,1.498329e-05,1,0,0_5
20935226,11,3.839994e-06,1,0,0_11
20935227,16,4.321017e-04,1,0,0_16
20935228,31,0.000000e+00,1,0,0_31
20935229,62,2.345477e-04,1,0,0_62
...,...,...,...,...,...
21148260,32736,6.959812e-04,60,0,0_32736
21148261,32739,5.730293e-04,60,0,0_32739
21148262,32748,7.492413e-05,60,0,0_32748
21148263,32753,2.130664e-07,60,0,0_32753


In [11]:
book_time[(book_time['stock_id']==0)& (book_time['time_id']==5)].head()

,time_id,sub_int_RV,sub_int_num,stock_id,row_id
20935225,5,0.000015,1,0,0_5
20938873,5,0.000010,2,0,0_5
20942455,5,0.001057,3,0,0_5
20949641,5,0.000898,5,0,0_5
20953221,5,0.000892,6,0,0_5


Is it expected that the time series for different stocks have a different number of rows? i.e., for 'stock_id'=1, we have 226527 rows:

In [12]:
book_time[book_time['stock_id']==1]

,time_id,sub_int_RV,sub_int_num,stock_id,row_id
8066266,5,0.000896,1,1,1_5
8066267,11,0.000288,1,1,1_11
8066268,16,0.000102,1,1,1_16
8066269,31,0.000895,1,1,1_31
8066270,62,0.000435,1,1,1_62
...,...,...,...,...,...
8292788,32751,0.000302,60,1,1_32751
8292789,32753,0.000631,60,1,1_32753
8292790,32758,0.000649,60,1,1_32758
8292791,32763,0.000822,60,1,1_32763
